# **Starbucks Offer Recommendation Engine for Customers**
---





In [ ]:
import pandas as pd
import numpy as np
import math
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Snehaa-path

In [ ]:
portfolio = pd.read_json('/content/drive/MyDrive/Marketing Analytics/ma_data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('/content/drive/MyDrive/Marketing Analytics/ma_data/profile.json', orient='records', lines=True)
transcript = pd.read_json('/content/drive/MyDrive/Marketing Analytics/ma_data/transcript.json', orient='records', lines=True)

Janet-path

In [ ]:
portfolio = pd.read_json('/content/drive/MyDrive/marketing_analytics/ma_package/ma_data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('/content/drive/MyDrive/marketing_analytics/ma_package/ma_data/profile.json', orient='records', lines=True)
transcript = pd.read_json('/content/drive/MyDrive/marketing_analytics/ma_package/ma_data/transcript.json', orient='records', lines=True)

In [ ]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [ ]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [ ]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


Data Cleaning

In [ ]:
#creating columns from portfolio data
portfolio["email"]=0
portfolio["mobile"]=0
portfolio["social"]=0
portfolio["web"]=0
portfolio["duration_hours"]=0

for index, row in portfolio.iterrows():
  if "email" in row["channels"]:
    portfolio.loc[index, "email"]=1
  if "mobile" in row["channels"]:
    portfolio.loc[index, "mobile"]=1
  if "web" in row["channels"]:
    portfolio.loc[index, "web"]=1
  if "social" in row["channels"]:
    portfolio.loc[index, "social"]=1
  portfolio.loc[index, "duration_hours"]=row["duration"]*24

portfolio.head()

,reward,channels,difficulty,duration,offer_type,id,email,mobile,social,web,duration_hours
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,168
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,120
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1,96
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,168
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,240


In [ ]:
offer_id_dict = dict(zip(portfolio["id"].tolist(), ["bogo1", "bogo2", "informational1", "bogo3", "discount1", "discount2", "discount3", "informational2", "bogo4", "discount4"]))
print(offer_id_dict)

{'ae264e3637204a6fb9bb56bc8210ddfd': 'bogo1', '4d5c57ea9a6940dd891ad53e9dbe8da0': 'bogo2', '3f207df678b143eea3cee63160fa8bed': 'informational1', '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'bogo3', '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'discount1', '2298d6c36e964ae4a3e7e9706d1fb8c2': 'discount2', 'fafdcd668e3743c1bb461111dcafc2a4': 'discount3', '5a8bc65990b245e5a138643cd4eb9837': 'informational2', 'f19421c1d4aa40978ebb69ca19b0e20d': 'bogo4', '2906b810c7d4411798c6938adc9daaa5': 'discount4'}


In [ ]:
portfolio["offer_type"].unique()

array(['bogo', 'informational', 'discount'], dtype=object)

In [ ]:
profile_transcript = pd.merge(profile, transcript, left_on="id", right_on="person")
profile_transcript.drop(["person"], axis=1, inplace=True)

profile_transcript["offer received"]=0
profile_transcript["offer viewed"]=0
profile_transcript["offer completed"]=0
profile_transcript["transaction"]=0
for index, row in profile_transcript.iterrows():
  if row["event"]=="offer received":
    profile_transcript.loc[index, "offer received"]=1
  if row["event"]=="offer viewed":
    profile_transcript.loc[index, "offer viewed"]=1
  if row["event"]=="offer received":
    profile_transcript.loc[index, "offer completed"]=1
  if row["event"]=="offer received":
    profile_transcript.loc[index, "transaction"]=1
    
profile_transcript.head()


,gender,age,id,became_member_on,income,event,value,time,offer received,offer viewed,offer completed,transaction
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},168,1,0,1,1
1,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,offer viewed,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},216,0,1,0,0
2,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},336,1,0,1,1
3,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,offer viewed,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},348,0,1,0,0
4,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,transaction,{'amount': 0.35000000000000003},360,0,0,0,0


--ignore--

In [ ]:
# profile_transcript = pd.merge(profile, transcript, left_on="id", right_on="person")
# profile_transcript.drop(["person"], axis=1, inplace=True)

# profile_transcript["offer received"]=0
# profile_transcript["offer viewed"]=0
# profile_transcript["offer completed"]=0
# profile_transcript["transaction"]=0
# for index, row in profile_transcript.iterrows():
#   profile_transcript.loc[index, row["event"]]=1
# profile_transcript.head()

KeyboardInterrupt: ignored

Putting profile_transcript into a csv file

In [ ]:
#profile_transcript.to_csv("/content/drive/MyDrive/marketing_analytics/ma_package/ma_data/profile_transcript.csv")  #janet
profile_transcript.to_csv("/content/drive/MyDrive/Marketing Analytics/ma_data/profile_transcript.csv")

In [ ]:
portfolio.to_csv("/content/drive/MyDrive/Marketing Analytics/ma_data/portfolio_updated.csv")

In [ ]:
profile.to_csv("/content/drive/MyDrive/marketing_analytics/ma_package/ma_data/profile_new.csv")